このnotebookは[ブログ記事](https://zenn.dev/erueru_tech/articles/ed8dec5e5cbda6)の内容を実行可能にするために用意したものとなる。

## 事前準備

macOS M3 silicon, VSCodeで動作確認。

この検証で使用するEconMLがPython3.13以上に対応していないので3.12を使用。

なおPythonのバージョン管理はpyenvで行なっていて、以下のコマンドでPythonのバージョンを指定する。

```bash
$ cd /path/to/oasobi
$ pyenv install --list
$ pyenv install 3.12.8
$ pyenv local 3.12.8
$ python -V
```

次に以下のコマンドでnotebook実行用の仮想環境構築。

```bash
$ python -m venv .venv.notebooks.vol2
$ source .venv.notebooks.vol2/bin/activate
$ pip install -r notebooks/vol2/requirements.txt
```

仮想環境を構築したらVSCode上で上記カーネルを選択。

## 検証

In [1]:
import numpy as np
import numpy.typing as npt
import pandas as pd
from econml.metalearners import TLearner
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
REPOSITORY_URL = "https://raw.githubusercontent.com/PacktPublishing/Causal-Inference-and-Discovery-in-Python/refs/heads/main/data"

### MineThatData データセット

In [3]:
hillstrom = pd.read_csv(f"{REPOSITORY_URL}/hillstrom_original.csv")
print("hillstrom.shape:", hillstrom.shape)
hillstrom.head()

hillstrom.shape: (64000, 12)


,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


MineThatDataデータセットに対して、zip_codeやchannelのワンホットエンコーディング、segmentの値を数値に変換といった前処理がすでに完了しているデータセットは以下。

なおsegmentはtreatmentという名前に変わっていて、値は No E-Mail -> 0, Womens E-Mail -> 1, Mens E-Mail -> 2 にそれぞれ変換されている。

In [4]:
hillstrom_clean = pd.read_csv(f"{REPOSITORY_URL}/hillstrom_clean.csv")
print("hillstrom_clean.shape:", hillstrom_clean.shape)
hillstrom_clean.head()

hillstrom_clean.shape: (64000, 15)


,recency,history,mens,womens,newbie,visit,conversion,spend,zip_code__rural,zip_code__surburban,zip_code__urban,channel__multichannel,channel__phone,channel__web,treatment
0,10,142.44,1,0,0,0,0,0.0,0,1,0,0,1,0,1
1,6,329.08,1,1,1,0,0,0.0,1,0,0,0,0,1,0
2,7,180.65,0,1,1,0,0,0.0,0,1,0,0,0,1,1
3,9,675.83,1,0,1,0,0,0.0,1,0,0,0,0,1,2
4,2,45.34,1,0,0,0,0,0.0,0,0,1,0,0,1,1


### ERMの計算式

以下の数式をPythonで実装。

$$Z = \sum_{t=0}^K \frac{1}{P_t}Y \mathbb{I}_{h(x)=t}\mathbb{I}_{T=t}$$

実装は書籍:[Pythonライブラリによる因果推論・因果探索[概念と実践] 因果機械学習の鍵を解く](https://book.impress.co.jp/books/1123101074)の[notebook](https://github.com/PacktPublishing/Causal-Inference-and-Discovery-in-Python/blob/main/Chapter_10.ipynb)を参考にした。

In [5]:
def get_expected_response(
    y_true: npt.NDArray[np.float64],
    t_true: npt.NDArray[np.int64],
    t_pred: npt.NDArray[np.int64],
) -> float:
    """アップリフトモデルのaverage expected responseを計算して返却する。
    proposed by:
    Zhao, Y., Fang, X., & Simchi-Levi, D. (2017). Uplift Modeling with Multiple Treatments and General Response Types.
    Proceedings of the 2017 SIAM International Conference on Data Mining, 588-596.
    Society for Industrial and Applied Mathematics.

    Args:
        y_true: 性能評価に使用するデータセットのアウトカムをあらわす列
        t_true: 性能評価に使用するデータセットの処置をあらわす列
        t_pred: モデルが予測した処置

    Returns:
        average expected response

    References:
        https://github.com/PacktPublishing/Causal-Inference-and-Discovery-in-Python/blob/main/Chapter_10.ipynb
    """
    proba_t = pd.Series(t_true).value_counts() / t_true.shape[0]
    treatments = proba_t.index.values

    z_vals = 0
    for treatment in treatments:
        h_indicator = t_pred == treatment
        t_indicator = t_true == treatment
        t_proba_local = proba_t[treatment]

        z_vals += (1 / t_proba_local) * y_true * h_indicator * t_indicator

    return z_vals.mean()

各処置の処置割り当て比率を計算する処理を抜き出して確認。

In [6]:
t_true = hillstrom_clean["treatment"]
pd.Series(t_true).value_counts() / t_true.shape[0]

treatment
1    0.334172
2    0.332922
0    0.332906
Name: count, dtype: float64

試しに以下のような条件のMineThatData仕様のダミーデータを作って、ERMの計算式の挙動を追ってみる。

- 各処置5人ずつ、合計15人の顧客を想定
- 処置:0では1/5人がコンバージョン、処置:1では2/5人がコンバージョン、処置:2では4/5人がコンバージョンが発生

In [7]:
def calc_expected_response(t_pred: npt.NDArray[np.int64]) -> None:
    # 各処置5人ずつ、合計15人の顧客を想定
    t_true = np.array([0, 0, 0, 0, 0] + [1, 1, 1, 1, 1] + [2, 2, 2, 2, 2])
    # 処置:0では1/5人がコンバージョン、処置:1では2/5人がコンバージョン、処置:2では4/5人がコンバージョンが発生
    y_true = np.array([1, 0, 0, 0, 0] + [1, 1, 0, 0, 0] + [1, 1, 1, 1, 0])
    # MineThatData仕様のダミーデータと引数で渡された任意の処置リストでERMスコアを計算
    expected_response = get_expected_response(y_true, t_true, t_pred)
    print(f"Expected Response: {expected_response:.4f}")

まず初めにMineThatData仕様のダミーデータの処置と、モデルが選択した処置リスト(t_pred)が完全に一致した場合のERMのスコアを確認してみる。

In [8]:
calc_expected_response(t_pred=np.array([0] * 5 + [1] * 5 + [2] * 5))

Expected Response: 1.4000


一方でMineThatData仕様のダミーデータの処置と、モデルが選択した処置リストが全く一致しなかった場合、もちろんスコアはゼロになる。

In [9]:
calc_expected_response(t_pred=np.array([2] * 5 + [0] * 5 + [1] * 5))

Expected Response: 0.0000


なお上記テストはテスト設計に問題がある。

実際のところ、評価データセットの処置とモデルが選択した処置が一致する確率は常に約33%になることが以下のコードで確認できる。

In [10]:
# 試しにproba_tに[0.50, 0.35, 0.15]や[1.00, 0.00, 0.00]などを指定しても、処置が一致する確率は常に約33%になる
proba_t = [0.33, 0.33, 0.34]
# モデルが選択した処置リストに見立てた値
t_pred = np.random.default_rng(0).choice(
    [0, 1, 2], size=hillstrom_clean.shape[0], p=proba_t
)
# 処置が一致した数 / データ総数
print((hillstrom_clean["treatment"] == t_pred).sum() / len(hillstrom_clean))

0.334125


以下のCTRはERMのスコア加算が行われる確率と一致する。

(詳細はブログ記事参照)

In [11]:
hillstrom_clean.groupby(["womens", "treatment"]).agg(
    cv_cnt=("conversion", "sum"),
    total_cnt=("conversion", "count"),
    cvr=("conversion", lambda x: x.sum() / x.count() if x.count() > 0 else 0),
).reset_index()

,womens,treatment,cv_cnt,total_cnt,cvr
0,0,0,53,9638,0.005499
1,0,1,59,9622,0.006132
2,0,2,110,9558,0.011509
3,1,0,69,11668,0.005914
4,1,1,130,11765,0.011050
5,1,2,157,11749,0.013363


In [12]:
# ちなみにwomens:0, mens:0のデータは存在しない
hillstrom_clean.groupby(["womens", "mens", "treatment"]).agg(
    cv_cnt=("conversion", "sum"),
    total_cnt=("conversion", "count"),
    cvr=("conversion", lambda x: x.sum() / x.count() if x.count() > 0 else 0),
).reset_index()

,womens,mens,treatment,cv_cnt,total_cnt,cvr
0,0,1,0,53,9638,0.005499
1,0,1,1,59,9622,0.006132
2,0,1,2,110,9558,0.011509
3,1,0,0,49,9519,0.005148
4,1,0,1,99,9647,0.010262
5,1,0,2,104,9568,0.010870
6,1,1,0,20,2149,0.009307
7,1,1,1,31,2118,0.014636
8,1,1,2,53,2181,0.024301


## ERMによるT-Learnerモデルとランダム処置割り当ての性能比較

以下の処理は基本的に書籍の[notebook](https://github.com/PacktPublishing/Causal-Inference-and-Discovery-in-Python/blob/main/Chapter_10.ipynb)を参考にしている。

In [13]:
# 多重共線性を考慮して冗長な列を削除
hillstrom_clean = hillstrom_clean.drop(["zip_code__urban", "channel__web"], axis=1)

# 特徴量、アウトカム、処置
hillstrom_X = hillstrom_clean.drop(
    ["visit", "conversion", "spend", "treatment"], axis=1
)
hillstrom_Y = hillstrom_clean["conversion"]
hillstrom_T = hillstrom_clean["treatment"]

# 学習用と評価用に半分に分割
X_train, X_test, Y_train, Y_test, T_train, T_test = train_test_split(
    hillstrom_X, hillstrom_Y, hillstrom_T, test_size=0.5
)

In [14]:
def create_model(
    random_state: int = 0,
    n_estimators: int = 100,
    max_depth: int = 10,
    learning_rate: float = 0.01,
) -> LGBMRegressor:
    return LGBMRegressor(
        random_state=random_state,
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
    )

In [15]:
t_learner = TLearner(models=[create_model() for _ in range(3)])
t_learner.fit(Y=Y_train, T=T_train, X=X_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000578 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 282
[LightGBM] [Info] Number of data points in the train set: 10680, number of used features: 9
[LightGBM] [Info] Start training from score 0.005150
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000116 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 282
[LightGBM] [Info] Number of data points in the train set: 10750, number of used features: 9
[LightGBM] [Info] Start training from score 0.009488
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000112 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2

In [16]:
def get_effects_argmax(
    effects_arrays: list[npt.NDArray[np.float64]], return_matrix: bool = False
) -> npt.NDArray[np.int64] | tuple[npt.NDArray[np.int64], npt.NDArray[np.float64]]:
    """各行における処置効果の一番大きい処置のインデックスを返す

    Args:
        effects_arrays: 統制群と各処置群の処置効果量の差をあらわす配列 # shape: (顧客数, 処置数(ただし統制群除く))
        return_matrix: 各行の処置効果量を完全に保持する行列を戻り値として返却するか

    Returns:
        各行の処置効果量最大の処置をあらわすインデックスの配列

    References:
        https://github.com/PacktPublishing/Causal-Inference-and-Discovery-in-Python/blob/main/Chapter_10.ipynb
    """

    n_rows = effects_arrays[0].shape[0]
    null_effect_array = np.zeros(n_rows)
    stacked = np.stack([null_effect_array] + effects_arrays).T

    if return_matrix:
        return np.argmax(stacked, axis=1), stacked

    return np.argmax(stacked, axis=1)

In [17]:
# 各顧客の特徴量に対する、処置0(No-Email)と処置1(Womens E-Mail)の予測処置効果量の差、
# 処置0と処置2(Mens E-Mail)の予測処置効果量の差を求めている
effects_argmax = get_effects_argmax(
    [t_learner.effect(X_train, T0=0, T1=t) for t in [1, 2]]
)

# T-Learnerモデルで処置を割り当てた場合のERMスコアを計算
expected_response = get_expected_response(
    y_true=Y_test, t_true=T_test, t_pred=effects_argmax
)
print(f"[T-Learner] Expected Response: {expected_response}")

# ランダムに処置を割り当てた場合のERMスコアを計算
expected_response = get_expected_response(
    y_true=Y_test,
    t_true=T_test,
    t_pred=np.random.default_rng(0).choice([0, 1, 2], size=len(T_test), p=proba_t),
)
print(f"[Random] Expected Response: {expected_response}")

[T-Learner] Expected Response: 0.011936667524715741
[Random] Expected Response: 0.008331025104548459
